## Imports

In [1]:
import nest_asyncio

nest_asyncio.apply()

import collections
import tensorflow as tf
import tensorflow_federated as tff
import pandas as pd
import numpy as np
import logging

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

from tqdm import tqdm

In [2]:
# Test the TFF is working:
tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

In [3]:
class HyperParameterPreprocessing:
  def __init__(self, prefetch_buffer_size, batch_size, num_epochs):
    self.prefetch_buffer_size =prefetch_buffer_size
    self.batch_size = batch_size
    self.num_epochs = num_epochs

In [4]:
class HyperParameterModel:
  def __init__(self, optimizer, learning_rate_client, learning_rate_server, rnn_units, num_rounds):
    self.learning_rate_client = learning_rate_client
    self.learning_rate_server = learning_rate_server
    self.optimizer = optimizer
    self.rnn_units = rnn_units
    self.num_rounds = num_rounds

In [5]:
logging.basicConfig(filename="./log/hyper_param_tuning/Evaluation.log", level=logging.INFO)

def log(text):
  print(text)
  logging.info(text)

In [6]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=False, batch_size=32):
  dataframe = df.copy()
  labels = dataframe.pop('cat_id')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [7]:
# List of numerical column names
numerical_column_names = ['clock_sin', 'clock_cos', 'day_sin', 'day_cos', 'month_sin', 'month_cos', 'week_day_sin', 'week_day_cos']

In [8]:
# Hyperparamters
HP_BATCH_SIZE = 8
HP_EMBEDDING = False
HP_LEARNING_RATE_CLIENT = 0.002
HP_LEARNING_RATE_SERVER = 0.06
HP_OPTIMIZER_CLIENT = tf.keras.optimizers.Adam(learning_rate=HP_LEARNING_RATE_CLIENT)
HP_OPTIMIZER_SERVER = tf.keras.optimizers.Adam(learning_rate=HP_LEARNING_RATE_SERVER)
HP_RNN_UNITS = 256


In [9]:
df = pd.read_csv("./4square/processed_transformed.csv")
df.head(100)

,user_id,cat_id,clock_sin,clock_cos,day_sin,day_cos,month_sin,month_cos,week_day_sin,week_day_cos
0,470,0,-1.000000,0.000654,0.587785,0.809017,0.866025,-0.500000,0.781831,0.623490
1,983,0,-0.242486,-0.970155,0.743145,0.669131,0.866025,-0.500000,0.974928,-0.222521
2,983,0,-0.400082,-0.916479,0.743145,0.669131,0.866025,-0.500000,0.974928,-0.222521
3,983,0,-0.449514,-0.893273,0.743145,0.669131,0.866025,-0.500000,0.974928,-0.222521
4,470,0,-1.000000,0.000654,0.743145,0.669131,0.866025,-0.500000,0.974928,-0.222521
...,...,...,...,...,...,...,...,...,...,...
95,31,0,-0.987287,0.158948,-0.743145,-0.669131,0.500000,-0.866025,-0.974928,-0.222521
96,596,0,-0.982301,-0.187310,-0.951057,-0.309017,0.500000,-0.866025,0.000000,1.000000
97,278,0,-0.836206,-0.548415,-0.994522,-0.104528,0.500000,-0.866025,0.781831,0.623490
98,596,0,-0.966076,-0.258257,-0.994522,0.104528,0.500000,-0.866025,0.974928,-0.222521


In [10]:
count = df.user_id.value_counts()

idx = count.loc[count.index[:100]].index # count >= 100
df = df.loc[df.user_id.isin(idx)]

In [11]:
# the number of different categories defines the vocabulary size
categories = df.cat_id
vocab_size = categories.nunique()

print('vocabulary size:', vocab_size)

vocabulary size: 27


In [12]:
# List the df for each user
users_locations = []

# For each user
for user_id in tqdm(idx):
  users_locations.append(df.loc[df.user_id == user_id].copy())

100%|██████████| 100/100 [00:00<00:00, 3199.80it/s]


In [13]:
# List the dfs fo train, val and test for each user
users_locations_train = []
users_locations_val = []
users_locations_test = []

for user_df in users_locations:
  # Split in train, test and validation
  train, test = train_test_split(user_df, test_size=0.2, shuffle=False)
  train, val = train_test_split(train, test_size=0.2, shuffle=False)

  # Append the sets
  users_locations_train.append(train)
  users_locations_val.append(val)
  users_locations_test.append(test)

In [14]:
sizes = []
# Number of locations for each user
for user_df in users_locations:
  sizes.append(user_df.shape[0])

print('Training')
print('Mean number of locations: ', np.mean(np.array(sizes)))
print('Max number of locations: ', np.max(np.array(sizes)))
print('Min number of locations: ', np.min(np.array(sizes)))

Training
Mean number of locations:  644.66
Max number of locations:  2697
Min number of locations:  387


In [15]:
sizes = []
# Number of locations for each user in the validation set
for user_df in users_locations_val:
  sizes.append(user_df.shape[0])

print('Validation')
print('Mean number of locations: ', np.mean(np.array(sizes)))
print('Max number of locations: ', np.max(np.array(sizes)))
print('Min number of locations: ', np.min(np.array(sizes)))

Validation
Mean number of locations:  103.48
Max number of locations:  432
Min number of locations:  62


In [16]:
sizes = []
# Number of locations for each user in the validation set
for user_df in users_locations_test:
  sizes.append(user_df.shape[0])

print('Mean number of locations: ', np.mean(np.array(sizes)))
print('Max number of locations: ', np.max(np.array(sizes)))
print('Min number of locations: ', np.min(np.array(sizes)))

Mean number of locations:  129.32
Max number of locations:  540
Min number of locations:  78


In [18]:
user_ids = df_train.user_id.unique()
print(user_ids.size)

100


In [19]:
columns_names = df_train.columns.values[1:]
print(columns_names)

['cat_id' 'clock_sin' 'clock_cos' 'day_sin' 'day_cos' 'month_sin'
 'month_cos' 'week_day_sin' 'week_day_cos']


In [20]:
NUM_CLIENTS = user_ids.size
#NUM_EPOCHS = 4
#PREFETCH_BUFFER = 5

In [21]:
# Split the data into chunks of N
def split_data(N):

  # dictionary of list of df
  df_dictionary = {}

  for uid in tqdm(user_ids):
    # Get the records of the user
    user_df_train = df_train.loc[df_train.user_id == uid].copy()
    user_df_val = df_val.loc[df_val.user_id == uid].copy()
    user_df_test = df_test.loc[df_test.user_id == uid].copy()

    # Get a list of dataframes of length N records
    user_list_train = [user_df_train[i:i+N] for i in range(0, user_df_train.shape[0], N)]
    user_list_val = [user_df_val[i:i+N] for i in range(0, user_df_val.shape[0], N)]
    user_list_test = [user_df_test[i:i+N] for i in range(0, user_df_test.shape[0], N)]

    # Save the list of dataframes into a dictionary
    df_dictionary[uid] = {
        'train': user_list_train,
        'val': user_list_val,
        'test': user_list_test
    }

  return  df_dictionary

In [22]:
# Takes a dictionary with train, validation and test sets and the desired set type
def create_clients_dict(df_dictionary, set_type, N):

  dataset_dict = {}

  for uid in tqdm(user_ids):

    c_data = collections.OrderedDict()
    values = df_dictionary[uid][set_type]

    # If the last dataframe of the list is not complete #
    if len(values[-1]) < N:
      diff = 1
    else:
      diff = 0

    if len(values) > 0:
      # Create the dictionary to create a clientData
      for header in columns_names:
        c_data[header] = [values[i][header].values for i in range(0, len(values)-diff)]
      dataset_dict[uid] = c_data

  return dataset_dict

In [23]:
# preprocess dataset to tf format
def preprocess(dataset, N, hyperparamters):

  NUM_EPOCHS = hyperparamters.num_epochs
  PREFETCH_BUFFER = hyperparamters.prefetch_buffer_size
  BATCH_SIZE = hyperparamters.batch_size

  def batch_format_fn(element):

    x=collections.OrderedDict()

    for name in columns_names:
      x[name]=tf.reshape(element[name][:, :-1], [-1, N-1])

    y=tf.reshape(element[columns_names[0]][:, 1:], [-1, N-1])

    return collections.OrderedDict(x=x, y=y)

  return dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE, drop_remainder=True).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

In [24]:
# create federated data for every client
def make_federated_data(client_data, client_ids, N, hyper_params_pre):

  return [
      preprocess(client_data.create_tf_dataset_for_client(x), N, hyper_params_pre)
      for x in tqdm(client_ids)
  ]

In [25]:
# Create a model
def create_keras_model(number_of_places, N, batch_size, embedding, rnn_units):

  # Shortcut to the layers package
  l = tf.keras.layers

  # List of numeric feature columns to pass to the DenseLayer
  numeric_feature_columns = []

  # Handling numerical columns
  for header in numerical_column_names:
		# Append all the numerical columns defined into the list
    numeric_feature_columns.append(feature_column.numeric_column(header, shape=N-1))

  feature_inputs={}
  for c_name in numerical_column_names:
    feature_inputs[c_name] = tf.keras.Input((N-1,), batch_size=batch_size, name=c_name)

  # We cannot use an array of features as always because we have sequences
  # We have to do one by one in order to match the shape
  num_features = []
  for c_name in numerical_column_names:
    f =  feature_column.numeric_column(c_name, shape=(N-1))
    feature = l.DenseFeatures(f)(feature_inputs)
    feature = tf.expand_dims(feature, -1)
    num_features.append(feature)

  # Declare the dictionary for the categories sequence as before #
  sequence_input = {
      'cat_id': tf.keras.Input((N-1,), batch_size=batch_size, dtype=tf.dtypes.int32, name='cat_id') # add batch_size=batch_size in case of stateful GRU
  }

  # Handling the categorical feature sequence using one-hot
  category_one_hot = feature_column.sequence_categorical_column_with_vocabulary_list(
      'cat_id', [i for i in range(number_of_places)])

  # category = feature_column.categorical_column_with_vocabulary_file(
  #    'cat_id', [str(i) for i in range(number_of_places)])

  category_encoded = feature_column.indicator_column(category_one_hot)


  if embedding:
    # Embed the one-hot encoding
    category_encoded = feature_column.embedding_column(category_one_hot, 256) # embedding_dim


  # With an input sequence we can't use the DenseFeature layer, we need to use the SequenceFeatures
  sequence_features, sequence_length = tf.keras.experimental.SequenceFeatures(category_encoded)(sequence_input)


  input_sequence = l.Concatenate(axis=2)([sequence_features] + num_features)

  # Rnn
  recurrent = l.GRU(rnn_units, # rnn_units
                        batch_size=batch_size, #in case of stateful
                        dropout=0.3,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform')(input_sequence)


	# Last layer with an output for each places
  dense_1 = layers.Dense(number_of_places)(recurrent)

	# Softmax output layer
  output = l.Softmax()(dense_1)

	# To return the Model, we need to define its inputs and outputs
	# In out case, we need to list all the input layers we have defined
  inputs = list(feature_inputs.values()) + list(sequence_input.values())

	# Return the Model
  return tf.keras.Model(inputs=inputs, outputs=output)

In [26]:
#train and evaluate the model
def train_and_eval_model(vocab_size, n, federated_train_data, federated_val_data, federated_test_data,  hyper_params_model, preprocessed_example_dataset, embedding_y_n, batch_size, path='./log/central-test-run'):
  train_logdir = path + '/train'
  val_logdir = path + '/val'
  eval_logdir = path + '/eval'

  num_rounds = hyper_params_model.num_rounds
  learning_rate_client = hyper_params_model.learning_rate_client
  learning_rate_server = hyper_params_model.learning_rate_server
  rnn_units = hyper_params_model.rnn_units
  optimizer = hyper_params_model.optimizer

  train_summary_writer = tf.summary.create_file_writer(train_logdir)
  val_summary_writer = tf.summary.create_file_writer(val_logdir)
  eval_summary_writer = tf.summary.create_file_writer(eval_logdir)

  # Clone the keras_model inside `create_tff_model()`, which TFF will
  # call to produce a new copy of the model inside the graph that it will
  # serialize. Note: we want to construct all the necessary objects we'll need
  # _inside_ this method.
  def create_tff_model():
    # TFF uses an `input_spec` so it knows the types and shapes
    # that your model expects.
    input_spec = preprocessed_example_dataset.element_spec
    keras_model_clone = create_keras_model(vocab_size, n, batch_size=batch_size, embedding=embedding_y_n, rnn_units=rnn_units)

    tff_model = tff.learning.from_keras_model(
      keras_model_clone,
      input_spec=input_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    return tff_model

  # This command builds all the TensorFlow graphs and serializes them:

  if optimizer == "Adam":
    fed_avg = tff.learning.build_federated_averaging_process(
    model_fn=create_tff_model,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=learning_rate_client),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=learning_rate_server))

  elif optimizer == "Nadam":
    fed_avg = tff.learning.build_federated_averaging_process(
    model_fn=create_tff_model,
    client_optimizer_fn=lambda: tf.keras.optimizers.Nadam(learning_rate=learning_rate_client),
    server_optimizer_fn=lambda: tf.keras.optimizers.Nadam(learning_rate=learning_rate_server))

  elif optimizer == "Adamax":
    fed_avg = tff.learning.build_federated_averaging_process(
    model_fn=create_tff_model,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adamax(learning_rate=learning_rate_client),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adamax(learning_rate=learning_rate_server))

  elif optimizer == "SGD":
    fed_avg = tff.learning.build_federated_averaging_process(
    model_fn=create_tff_model,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=learning_rate_client),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=learning_rate_server))

  elif optimizer == "Adadelta":
    fed_avg = tff.learning.build_federated_averaging_process(
    model_fn=create_tff_model,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adadelta(learning_rate=learning_rate_client),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adadelta(learning_rate=learning_rate_server))

  elif optimizer == "Adagrad":
    fed_avg = tff.learning.build_federated_averaging_process(
    model_fn=create_tff_model,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adagrad(learning_rate=learning_rate_client),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adagrad(learning_rate=learning_rate_server))

  elif optimizer == "Ftrl":
    fed_avg = tff.learning.build_federated_averaging_process(
    model_fn=create_tff_model,
    client_optimizer_fn=lambda: tf.keras.optimizers.Ftrl(learning_rate=learning_rate_client),
    server_optimizer_fn=lambda: tf.keras.optimizers.Ftrl(learning_rate=learning_rate_server))

  elif optimizer == "RMSprop":
    fed_avg = tff.learning.build_federated_averaging_process(
    model_fn=create_tff_model,
    client_optimizer_fn=lambda: tf.keras.optimizers.RMSprop(learning_rate=learning_rate_client),
    server_optimizer_fn=lambda: tf.keras.optimizers.RMSprop(learning_rate=learning_rate_server))


  state = fed_avg.initialize()
  evaluation = tff.learning.build_federated_evaluation(model_fn=create_tff_model)
  val_metrics = evaluation(state.model, federated_val_data)

  tolerance = 7
  best_state = 0
  lowest_loss = 100.00
  stop = tolerance

  NUM_ROUNDS = 10 #40
  with train_summary_writer.as_default():
    for round_num in range(1, num_rounds + 1):
      print('Round {r}'.format(r=round_num))

      # Uncomment to simulate sparse availability of clients
      # train_data_for_this_round, val_data_for_this_round = sample((federated_train_data, federated_val_data), 20, NUM_CLIENTS)

      state, metrics = fed_avg.next(state, federated_train_data)

      train_metrics = metrics['train']
      print('\tTrain: loss={l:.3f}, accuracy={a:.3f}'.format(l=train_metrics['loss'], a=train_metrics['sparse_categorical_accuracy']))

      val_metrics = evaluation(state.model, federated_val_data)
      print('\tValidation: loss={l:.3f}, accuracy={a:.3f}'.format( l=val_metrics['loss'], a=val_metrics['sparse_categorical_accuracy']))

      # Check for decreasing validation loss
      if lowest_loss > val_metrics['loss']:
        print('\tSaving best model..')
        lowest_loss = val_metrics['loss']
        best_state = state
        stop = tolerance - 1
      else:
        stop = stop - 1
        if stop <= 0:
          print('\tEarly stopping...')
          break;

      print(' ')
      print('\twriting..')

      # Iterate across the metrics and write their data
      for name, value in dict(train_metrics).items():
        tf.summary.scalar('epoch_'+name, value, step=round_num)

      with val_summary_writer.as_default():
        for name, value in dict(val_metrics).items():
          tf.summary.scalar('epoch_'+name, value, step=round_num)

  train_summary_writer.close()
  val_summary_writer.close()

  # evaluate over test data
  test_metrics = evaluation(best_state.model, federated_test_data)
  log('\tEvaluation: loss={l:.3f}, accuracy={a:.3f}'.format( l=test_metrics['loss'], a=test_metrics['sparse_categorical_accuracy']))
  return test_metrics

In [27]:
n=17
print('Splitting data...')
df_dict = split_data(n)
print('Creating client dictionaries...')
clients_train_dict = create_clients_dict(df_dict, 'train', n)
clients_val_dict = create_clients_dict(df_dict, 'val', n)
clients_test_dict = create_clients_dict(df_dict, 'test', n)

# Convert the dictionary to a dataset
print('Converting to datasets...')
client_train_data = tff.simulation.FromTensorSlicesClientData(clients_train_dict)
client_val_data = tff.simulation.FromTensorSlicesClientData(clients_val_dict)
client_test_data = tff.simulation.FromTensorSlicesClientData(clients_test_dict)

Splitting data...


100%|██████████| 100/100 [00:00<00:00, 711.29it/s]


Creating client dictionaries...


100%|██████████| 100/100 [00:00<00:00, 735.81it/s]

Converting to datasets...


In [28]:
def execute_hyper_tuning(hyper_params_pre, hyper_params_model, embedding_y_n):
  example_dataset = client_train_data.create_tf_dataset_for_client(client_train_data.client_ids[1])
  preprocessed_example_dataset = preprocess(example_dataset, n, hyper_params_pre)

  # Select the clients
  sample_clients = client_train_data.client_ids[0:NUM_CLIENTS]

  # Federate the clients datasets
  print('Federating datasets...')
  federated_train_data = make_federated_data(client_train_data, sample_clients, n, hyper_params_pre)
  federated_val_data = make_federated_data(client_val_data, sample_clients, n, hyper_params_pre)
  federated_test_data = make_federated_data(client_test_data, sample_clients, n, hyper_params_pre)

  print('Training...')
  p="'./log/central-test-run_%d" % n
  eval_result=train_and_eval_model(vocab_size, n, federated_train_data, federated_val_data, federated_test_data, path=p, hyper_params_model=hyper_params_model, preprocessed_example_dataset=preprocessed_example_dataset, embedding_y_n=embedding_y_n, batch_size=hyper_params_pre.batch_size)
  return eval_result

In [29]:
#Hyperparamters
#Embedding vs One-Hot Encoding
embeddings = [True, False]
#Preprocessing Parameters
prefetch_buffer_sizes = [2, 4, 8]
batch_sizes = [8, 16, 32]
num_epochs = [2, 5, 7]
#Model Parameters
optimizers = ["Adam", "Nadam", "Adamax"]
learning_rates_server = [0.06, 0.6, 0.006]
learning_rates_client = [0.001, 0.002, 0.003]
num_rnn_units = [64, 128, 256]
num_rounds = 2

parameter_arr_pre = [prefetch_buffer_sizes, batch_sizes, num_epochs]
parameter_arr_model = [optimizers, learning_rates_server, learning_rates_client, num_rnn_units]

In [30]:
eval_results_embedding = []
def best_hyper_param(result_array):
  best_loss = -1
  best_accuracy = -1
  best_param = -1
  i = 0
  for result in result_array:
    l=result[0]['loss']
    a=result[0]['sparse_categorical_accuracy']
    if l < best_loss and a > best_accuracy:
      best_loss = l
      best_accuracy = a
      best_param = i
      i += 1
  return best_param

#Test Embedding vs One-Hot
for embedding in embeddings:
   hyper_params_pre = HyperParameterPreprocessing(parameter_arr_pre[0][0],parameter_arr_pre[1][0],parameter_arr_pre[2][0])
   hyper_params_model = HyperParameterModel(parameter_arr_model[0][0], parameter_arr_model[1][0], parameter_arr_model[2][0], parameter_arr_model[3][0], num_rounds=num_rounds)
   log('Run for rounds=%d, \n Preprocessing: prefetch_buffer=%d; batch_size=%d; num_epochs=%d;'
       '\n Model: optimizer:%s; learning_rate_server=%f; learning_rate_client=%f; rnn_units=%d' % (num_rounds, hyper_params_pre.prefetch_buffer_size, hyper_params_pre.batch_size, hyper_params_pre.num_epochs, hyper_params_model.optimizer, hyper_params_model.learning_rate_server, hyper_params_model.learning_rate_client, hyper_params_model.rnn_units))

   eval_result=execute_hyper_tuning(hyper_params_pre=hyper_params_pre, hyper_params_model=hyper_params_model, embedding_y_n=embedding)
   eval_results_embedding.append([eval_result])

embedding_y_n_result=best_hyper_param(eval_results_embedding)
embedding_y_n = embeddings[embedding_y_n_result]
log('Embedding better than One-Hot?: %r' % embedding_y_n)

eval_results_pre = []

p = 0
b = 0
e = 0
for p_size in prefetch_buffer_sizes:
  for b_size in batch_sizes:
    for number in num_epochs:
      hyper_params_pre = HyperParameterPreprocessing(p_size,b_size,number)
      hyper_params_model = HyperParameterModel(parameter_arr_model[0][0], parameter_arr_model[1][0], parameter_arr_model[2][0], parameter_arr_model[3][0], num_rounds=num_rounds)
      log('Run for rounds=%d, \n Preprocessing: prefetch_buffer=%d; batch_size=%d; num_epochs=%d;'
          '\n Model: optimizer:%s; learning_rate_server=%f; learning_rate_client=%f; rnn_units=%d' % (num_rounds, hyper_params_pre.prefetch_buffer_size, hyper_params_pre.batch_size, hyper_params_pre.num_epochs, hyper_params_model.optimizer, hyper_params_model.learning_rate_server, hyper_params_model.learning_rate_client, hyper_params_model.rnn_units))
      eval_result=execute_hyper_tuning(hyper_params_pre=hyper_params_pre, hyper_params_model=hyper_params_model, embedding_y_n=embedding_y_n)
      eval_results_pre.append([eval_result, [p,b,e]])

      e+=1
    b+=1
  p+=1

best_prep_combination=best_hyper_param(eval_results_pre)
best_prefetch_size = eval_results_pre[best_prep_combination][0]
best_buffer_size = eval_results_pre[best_prep_combination][1]
best_num_epochs = eval_results_pre[best_prep_combination][2]
log('Best combination for preprocessing: prefetch_size:%d; buffer_size:%d; num_epochs:%d' % (best_prefetch_size, best_buffer_size, best_num_epochs))

Run for rounds=2, 
 Preprocessing: prefetch_buffer=2; batch_size=8; num_epochs=2;
 Model: optimizer:Adam; learning_rate_server=0.001000; learning_rate_client=0.060000; rnn_units=64
Federating datasets...


100%|██████████| 100/100 [00:02<00:00, 33.76it/s]


Training...
Round 1
	Train: loss=2.388, accuracy=0.416
	Validation: loss=3.261, accuracy=0.073
	Saving best model..
 
	writing..
Round 2
	Train: loss=2.352, accuracy=0.447
	Validation: loss=3.231, accuracy=0.105
	Saving best model..
 
	writing..
	Evaluation: loss=3.248, accuracy=0.071
Run for rounds=2, 
 Preprocessing: prefetch_buffer=2; batch_size=8; num_epochs=2;
 Model: optimizer:Adam; learning_rate_server=0.001000; learning_rate_client=0.060000; rnn_units=64
Federating datasets...


100%|██████████| 100/100 [00:03<00:00, 30.28it/s]


Training...
Round 1
	Train: loss=2.951, accuracy=0.260
	Validation: loss=3.289, accuracy=0.025
	Saving best model..
 
	writing..
Round 2
	Train: loss=2.940, accuracy=0.262
	Validation: loss=3.277, accuracy=0.030
	Saving best model..
 
	writing..
	Evaluation: loss=3.298, accuracy=0.025
Embedding better than One-Hot?: False
Run for rounds=2, 
 Preprocessing: prefetch_buffer=2; batch_size=8; num_epochs=2;
 Model: optimizer:Adam; learning_rate_server=0.001000; learning_rate_client=0.060000; rnn_units=64
Federating datasets...


...

100%|██████████| 100/100 [00:03<00:00, 31.43it/s]


Training...
Round 1
	Train: loss=3.108, accuracy=0.130
	Validation: loss=3.296, accuracy=0.051
	Saving best model..
 
	writing..
Round 2
	Train: loss=3.077, accuracy=0.134
	Validation: loss=3.278, accuracy=0.053
	Saving best model..
 
	writing..
	Evaluation: loss=3.182, accuracy=0.191
Run for rounds=2, 
 Preprocessing: prefetch_buffer=4; batch_size=32; num_epochs=5;
 Model: optimizer:Adam; learning_rate_server=0.001000; learning_rate_client=0.060000; rnn_units=64
Federating datasets...


100%|██████████| 100/100 [00:09<00:00, 10.94it/s]


Training...


KeyboardInterrupt: 

In [ ]:
eval_results_model = []

o = 0
s = 0
c = 0
r = 0
for optimizer in optimizers:
  for lr_server in learning_rates_client:
    for lr_client in learning_rates_server:
      for rnn_units in num_rnn_units:
        hyper_params_pre = HyperParameterPreprocessing(best_prefetch_size,best_buffer_size,best_num_epochs)
        hyper_params_model = HyperParameterModel(optimizer=optimizer, learning_rate_server=lr_server, learning_rate_client=lr_client,rnn_units=rnn_units, num_rounds=num_rounds)
        log('Run for rounds=%d, \n Preprocessing: prefetch_buffer=%d; batch_size=%d; num_epochs=%d;'
          '\n Model: optimizer:%s; learning_rate_server=%f; learning_rate_client=%f; rnn_units=%d' % (num_rounds, hyper_params_pre.prefetch_buffer_size, hyper_params_pre.batch_size, hyper_params_pre.num_epochs, hyper_params_model.optimizer, hyper_params_model.learning_rate_server, hyper_params_model.learning_rate_client, hyper_params_model.rnn_units))
        eval_result=execute_hyper_tuning(hyper_params_pre=hyper_params_pre, hyper_params_model=hyper_params_model, embedding_y_n=embedding_y_n)
        eval_results_model.append([eval_result, [o,s,c,r]])
        r+=1
      c+=1
    s+=1
  r+=1

best_model_combination=best_hyper_param(eval_results_model)
best_optimizer = eval_results_model[best_model_combination][0]
best_learning_rate_server = eval_results_model[best_model_combination][1]
best_learning_rate_client = eval_results_model[best_model_combination][2]
best_rnn_units = eval_results_model[best_model_combination][3]
log('Best combination for the model: optimizer:%s; learning_rate_server=%f; learning_rate_client=%f; rnn_units=%d' % (best_optimizer, best_learning_rate_server, best_learning_rate_client, best_rnn_units))

In [ ]:
#for rounds in num_rounds:
#  for embedding in embeddings:
#    for batch_size in batch_sizes:
#      for rnn_units in num_rnn_units:
#        for learning_rate_client in learning_rates_client:
#          for learning_rate_server in learning_rates_server:
#            for optimizer in optimizers:
#              hyper_params = HyperParameter(num_rounds=rounds, batch_size=batch_size, rnn_units=rnn_units, learning_rate_client=learning_rate_client, learning_rate_server=learning_rate_server, embedding=embedding, optimizer=optimizer)
#              log('Run for rounds=%d, batch_size=%d, rnn_units=%d, learning_rate_client=%f, learning_rate_server=%f' % (rounds, batch_size, rnn_units, learning_rate_client, learning_rate_server))
#              execute_hyper_tuning(hyper_params=hyper_params)